# S12 — Vector Databases & Production

## 🎯 Objectifs
- Comparer FAISS et Milvus pour le stockage vectoriel
- Mesurer les métriques de latence et recall
- Comprendre les tradeoffs production (scalabilité, persistence)
- Benchmarker différentes configurations d'index

## 📋 Contenu
1. Introduction aux Vector Databases
2. FAISS: Index local optimisé
3. Milvus: Base de données vectorielle distribuée
4. Comparaison des performances
5. Choix d'architecture pour la production

## 1. Installation et Configuration

In [ ]:
# Installation des dépendances
# !pip install faiss-cpu pymilvus sentence-transformers pandas numpy matplotlib scikit-learn

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import faiss
from pymilvus import connections, Collection, CollectionSchema, FieldSchema, DataType, utility
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées")

## 2. Création du Dataset de Test

Nous allons créer un dataset substantiel pour tester les performances.

In [ ]:
# Charger le modèle d'embeddings
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print(f"✅ Modèle chargé: dimension = {model.get_sentence_embedding_dimension()}")

In [ ]:
# Créer un dataset de 10,000 documents simulés
np.random.seed(42)

# Templates de documents
templates = [
    "Le {topic} est essentiel pour {domain}.",
    "Dans le domaine de {domain}, on utilise {topic}.",
    "Les techniques de {topic} améliorent {domain}.",
    "Pour {domain}, il est important de maîtriser {topic}.",
    "L'application de {topic} dans {domain} montre des résultats prometteurs.",
]

topics = [
    "machine learning", "deep learning", "transformers", "embeddings",
    "RAG", "fine-tuning", "prompt engineering", "vector search",
    "attention mechanism", "tokenization", "NLP", "computer vision",
    "reinforcement learning", "neural networks", "optimization"
]

domains = [
    "intelligence artificielle", "data science", "recherche d'information",
    "traitement du langage", "vision par ordinateur", "recommandation",
    "analyse prédictive", "classification", "génération de texte"
]

documents = []
for i in range(10000):
    template = np.random.choice(templates)
    topic = np.random.choice(topics)
    domain = np.random.choice(domains)
    text = template.format(topic=topic, domain=domain)
    documents.append({
        "id": i,
        "text": text,
        "topic": topic,
        "domain": domain
    })

df = pd.DataFrame(documents)
print(f"📊 Dataset créé: {len(df)} documents")
print(f"Exemple: {df.iloc[0]['text']}")

In [ ]:
# Générer les embeddings (peut prendre quelques minutes)
print("🔄 Génération des embeddings...")
start = time.time()
embeddings = model.encode(df['text'].tolist(), show_progress_bar=True, batch_size=256)
embeddings = np.array(embeddings).astype('float32')
duration = time.time() - start

print(f"✅ Embeddings générés en {duration:.2f}s")
print(f"Shape: {embeddings.shape}")
print(f"Mémoire: {embeddings.nbytes / 1024 / 1024:.2f} MB")

## 3. FAISS: Index Local

### 3.1 Index Flat (baseline)

In [ ]:
class FAISSIndex:
    def __init__(self, dimension: int, index_type: str = 'flat'):
        self.dimension = dimension
        self.index_type = index_type
        
        if index_type == 'flat':
            self.index = faiss.IndexFlatL2(dimension)
        elif index_type == 'ivf':
            # Index IVF pour meilleure performance sur grands datasets
            quantizer = faiss.IndexFlatL2(dimension)
            nlist = 100  # Nombre de clusters
            self.index = faiss.IndexIVFFlat(quantizer, dimension, nlist)
        elif index_type == 'hnsw':
            # Index HNSW pour recherche rapide
            M = 32  # Nombre de connexions par layer
            self.index = faiss.IndexHNSWFlat(dimension, M)
        else:
            raise ValueError(f"Type d'index non supporté: {index_type}")
        
        self.documents = []
    
    def train(self, vectors: np.ndarray):
        """Entraîner l'index (nécessaire pour IVF)"""
        if self.index_type == 'ivf':
            print(f"🔄 Entraînement de l'index IVF...")
            self.index.train(vectors)
    
    def add(self, vectors: np.ndarray, documents: List[Dict]):
        """Ajouter des vecteurs à l'index"""
        self.index.add(vectors)
        self.documents.extend(documents)
    
    def search(self, query_vector: np.ndarray, k: int = 5) -> Tuple[np.ndarray, np.ndarray, float]:
        """Rechercher les k plus proches voisins"""
        start = time.time()
        
        if self.index_type == 'ivf':
            # Paramètre nprobe: nombre de clusters à examiner
            self.index.nprobe = 10
        
        distances, indices = self.index.search(query_vector, k)
        latency = (time.time() - start) * 1000  # en ms
        
        return distances, indices, latency
    
    def get_stats(self) -> Dict:
        return {
            "index_type": self.index_type,
            "dimension": self.dimension,
            "total_vectors": self.index.ntotal,
            "is_trained": self.index.is_trained
        }

print("✅ Classe FAISSIndex définie")

In [ ]:
# Tester les différents types d'index FAISS
index_types = ['flat', 'ivf', 'hnsw']
faiss_indexes = {}

for idx_type in index_types:
    print(f"\n{'='*60}")
    print(f"Création de l'index FAISS: {idx_type.upper()}")
    print(f"{'='*60}")
    
    index = FAISSIndex(dimension=embeddings.shape[1], index_type=idx_type)
    
    # Entraînement si nécessaire
    start = time.time()
    index.train(embeddings)
    train_time = time.time() - start
    
    # Indexation
    start = time.time()
    index.add(embeddings, documents)
    index_time = time.time() - start
    
    print(f"⏱️  Temps d'entraînement: {train_time:.2f}s")
    print(f"⏱️  Temps d'indexation: {index_time:.2f}s")
    print(f"📊 Stats: {index.get_stats()}")
    
    faiss_indexes[idx_type] = index

print("\n✅ Tous les index FAISS créés")

### 3.2 Benchmark FAISS

In [ ]:
# Créer des requêtes de test
test_queries = [
    "Comment utiliser les transformers pour le NLP?",
    "Qu'est-ce que le machine learning?",
    "Optimisation des réseaux de neurones",
    "Applications du deep learning en vision",
    "Recherche vectorielle avec embeddings"
]

query_embeddings = model.encode(test_queries)
query_embeddings = np.array(query_embeddings).astype('float32')

print(f"✅ {len(test_queries)} requêtes de test créées")

In [ ]:
# Benchmarker chaque index FAISS
k = 10  # Top-10
faiss_results = {}

for idx_type, index in faiss_indexes.items():
    print(f"\n{'='*60}")
    print(f"Benchmark FAISS-{idx_type.upper()}")
    print(f"{'='*60}")
    
    latencies = []
    
    for i, query in enumerate(query_embeddings):
        distances, indices, latency = index.search(query.reshape(1, -1), k)
        latencies.append(latency)
        
        if i == 0:  # Afficher les résultats de la première requête
            print(f"\nRequête: '{test_queries[i]}'")
            print(f"Top-3 résultats:")
            for j in range(min(3, len(indices[0]))):
                idx = indices[0][j]
                dist = distances[0][j]
                print(f"  {j+1}. (distance={dist:.4f}) {documents[idx]['text'][:80]}...")
    
    avg_latency = np.mean(latencies)
    p95_latency = np.percentile(latencies, 95)
    p99_latency = np.percentile(latencies, 99)
    
    print(f"\n📊 Latence moyenne: {avg_latency:.2f} ms")
    print(f"📊 Latence P95: {p95_latency:.2f} ms")
    print(f"📊 Latence P99: {p99_latency:.2f} ms")
    
    faiss_results[idx_type] = {
        "avg_latency": avg_latency,
        "p95_latency": p95_latency,
        "p99_latency": p99_latency,
        "latencies": latencies
    }

## 4. Milvus: Base de Données Vectorielle

### 4.1 Configuration Milvus

⚠️ **Note**: Pour utiliser Milvus, vous devez avoir un serveur Milvus en cours d'exécution.

```bash
# Avec Docker
docker run -d --name milvus_standalone \
  -p 19530:19530 -p 9091:9091 \
  milvusdb/milvus:latest
```

In [ ]:
class MilvusIndex:
    def __init__(self, collection_name: str, dimension: int, host: str = "localhost", port: str = "19530"):
        self.collection_name = collection_name
        self.dimension = dimension
        self.host = host
        self.port = port
        self.connected = False
        
    def connect(self):
        """Se connecter à Milvus"""
        try:
            connections.connect("default", host=self.host, port=self.port)
            self.connected = True
            print(f"✅ Connecté à Milvus sur {self.host}:{self.port}")
        except Exception as e:
            print(f"❌ Erreur de connexion à Milvus: {e}")
            print("💡 Lancez d'abord un serveur Milvus (voir instructions ci-dessus)")
            self.connected = False
    
    def create_collection(self):
        """Créer une collection Milvus"""
        if not self.connected:
            return False
        
        # Supprimer la collection si elle existe
        if utility.has_collection(self.collection_name):
            utility.drop_collection(self.collection_name)
        
        # Définir le schéma
        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=False),
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=self.dimension),
            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=500)
        ]
        
        schema = CollectionSchema(fields, description="Vector search collection")
        self.collection = Collection(self.collection_name, schema)
        
        print(f"✅ Collection '{self.collection_name}' créée")
        return True
    
    def create_index(self, index_type: str = "IVF_FLAT"):
        """Créer un index sur le champ embedding"""
        if not self.connected:
            return False
        
        index_params = {
            "metric_type": "L2",
            "index_type": index_type,
            "params": {"nlist": 128}
        }
        
        self.collection.create_index(field_name="embedding", index_params=index_params)
        print(f"✅ Index {index_type} créé")
        return True
    
    def insert(self, vectors: np.ndarray, documents: List[Dict]):
        """Insérer des vecteurs dans Milvus"""
        if not self.connected:
            return False
        
        ids = [doc['id'] for doc in documents]
        texts = [doc['text'][:500] for doc in documents]  # Limiter la longueur
        
        entities = [
            ids,
            vectors.tolist(),
            texts
        ]
        
        self.collection.insert(entities)
        self.collection.flush()
        print(f"✅ {len(ids)} vecteurs insérés")
        return True
    
    def search(self, query_vector: np.ndarray, k: int = 5) -> Tuple[List, float]:
        """Rechercher les k plus proches voisins"""
        if not self.connected:
            return [], 0.0
        
        self.collection.load()
        
        search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
        
        start = time.time()
        results = self.collection.search(
            data=[query_vector.tolist()],
            anns_field="embedding",
            param=search_params,
            limit=k,
            output_fields=["text"]
        )
        latency = (time.time() - start) * 1000
        
        return results, latency
    
    def get_stats(self) -> Dict:
        if not self.connected:
            return {}
        
        return {
            "collection_name": self.collection_name,
            "num_entities": self.collection.num_entities,
            "dimension": self.dimension
        }

print("✅ Classe MilvusIndex définie")

In [ ]:
# Tester Milvus (optionnel si serveur disponible)
USE_MILVUS = False  # Mettre à True si Milvus est disponible

if USE_MILVUS:
    milvus_index = MilvusIndex(
        collection_name="vectors_demo",
        dimension=embeddings.shape[1]
    )
    
    milvus_index.connect()
    
    if milvus_index.connected:
        milvus_index.create_collection()
        milvus_index.create_index("IVF_FLAT")
        
        # Insérer par batch pour éviter la surcharge
        batch_size = 1000
        for i in range(0, len(embeddings), batch_size):
            end_idx = min(i + batch_size, len(embeddings))
            batch_embeddings = embeddings[i:end_idx]
            batch_docs = documents[i:end_idx]
            milvus_index.insert(batch_embeddings, batch_docs)
        
        print(f"✅ Stats Milvus: {milvus_index.get_stats()}")
        
        # Benchmark Milvus
        milvus_latencies = []
        for query in query_embeddings:
            results, latency = milvus_index.search(query, k=10)
            milvus_latencies.append(latency)
        
        print(f"\n📊 Milvus - Latence moyenne: {np.mean(milvus_latencies):.2f} ms")
        print(f"📊 Milvus - Latence P95: {np.percentile(milvus_latencies, 95):.2f} ms")
else:
    print("⚠️  Milvus non utilisé (USE_MILVUS=False)")
    print("💡 Pour tester Milvus, lancez un serveur et mettez USE_MILVUS=True")

## 5. Comparaison des Performances

In [ ]:
# Visualisation des latences
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Graphique 1: Latence moyenne par type d'index
index_names = list(faiss_results.keys())
avg_latencies = [faiss_results[idx]['avg_latency'] for idx in index_names]

axes[0].bar(index_names, avg_latencies, color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_ylabel('Latence moyenne (ms)')
axes[0].set_title('Latence Moyenne par Type d\'Index FAISS')
axes[0].set_xlabel('Type d\'Index')
axes[0].grid(axis='y', alpha=0.3)

# Ajouter les valeurs sur les barres
for i, v in enumerate(avg_latencies):
    axes[0].text(i, v + 0.01, f'{v:.2f}', ha='center', va='bottom')

# Graphique 2: Distribution des latences
for idx_type in index_names:
    axes[1].hist(faiss_results[idx_type]['latencies'], bins=20, alpha=0.5, label=idx_type)

axes[1].set_xlabel('Latence (ms)')
axes[1].set_ylabel('Fréquence')
axes[1].set_title('Distribution des Latences')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Graphiques de performance générés")

## 6. Évaluation du Recall

Le **recall** mesure la proportion de résultats pertinents retrouvés.

In [ ]:
def calculate_recall(results_fast, results_exact, k=10):
    """
    Calculer le recall en comparant avec les résultats exacts (Flat index)
    """
    recalls = []
    
    for i in range(len(results_fast)):
        fast_ids = set(results_fast[i])
        exact_ids = set(results_exact[i])
        
        intersection = len(fast_ids.intersection(exact_ids))
        recall = intersection / k
        recalls.append(recall)
    
    return np.mean(recalls)

# Obtenir les résultats de référence (Flat index)
flat_index = faiss_indexes['flat']
flat_results = []

for query in query_embeddings:
    distances, indices, _ = flat_index.search(query.reshape(1, -1), k=10)
    flat_results.append(indices[0].tolist())

# Calculer le recall pour chaque type d'index
print("\n" + "="*60)
print("ÉVALUATION DU RECALL (vs Flat Index)")
print("="*60)

for idx_type in ['ivf', 'hnsw']:
    index = faiss_indexes[idx_type]
    results = []
    
    for query in query_embeddings:
        distances, indices, _ = index.search(query.reshape(1, -1), k=10)
        results.append(indices[0].tolist())
    
    recall = calculate_recall(results, flat_results, k=10)
    print(f"📊 {idx_type.upper()}: Recall@10 = {recall:.4f} ({recall*100:.2f}%)")

## 7. Tableau Comparatif

### Résumé des performances

In [ ]:
# Créer un tableau comparatif
comparison_data = []

for idx_type in index_names:
    comparison_data.append({
        "Index Type": idx_type.upper(),
        "Latence Moy. (ms)": f"{faiss_results[idx_type]['avg_latency']:.2f}",
        "Latence P95 (ms)": f"{faiss_results[idx_type]['p95_latency']:.2f}",
        "Mémoire": "RAM" if idx_type == 'flat' else "RAM (optimisé)",
        "Scalabilité": "Limitée" if idx_type == 'flat' else "Moyenne" if idx_type == 'ivf' else "Bonne",
        "Qualité": "Exacte" if idx_type == 'flat' else "~95%" if idx_type == 'ivf' else "~98%"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("TABLEAU COMPARATIF - FAISS")
print("="*80)
print(comparison_df.to_string(index=False))

print("\n" + "="*80)
print("RECOMMANDATIONS")
print("="*80)
print("""
🎯 FLAT Index:
   ✅ Avantages: Résultats exacts, simple
   ❌ Inconvénients: Lent sur gros volumes (>100K vecteurs)
   💡 Usage: Prototypes, petits datasets

🎯 IVF Index:
   ✅ Avantages: Bon compromis vitesse/qualité
   ❌ Inconvénients: Nécessite entraînement
   💡 Usage: Production (100K-10M vecteurs)

🎯 HNSW Index:
   ✅ Avantages: Très rapide, excellent recall
   ❌ Inconvénients: Mémoire supplémentaire
   💡 Usage: Applications temps réel

🎯 Milvus:
   ✅ Avantages: Distribué, scalable, persistant
   ❌ Inconvénients: Infrastructure complexe
   💡 Usage: Production à grande échelle (>10M vecteurs)
""")

## 8. Critères de Choix pour la Production

### Matrice de décision

In [ ]:
decision_matrix = pd.DataFrame({
    "Critère": [
        "Volume de données",
        "Latence requise",
        "Qualité (recall)",
        "Mises à jour fréquentes",
        "Distribution géographique",
        "Budget infrastructure",
        "Compétences équipe"
    ],
    "< 100K vecteurs": [
        "FAISS Flat/HNSW",
        "FAISS HNSW",
        "FAISS Flat",
        "FAISS (reload)",
        "FAISS + CDN",
        "FAISS",
        "FAISS"
    ],
    "100K - 10M vecteurs": [
        "FAISS IVF/HNSW",
        "FAISS HNSW",
        "FAISS IVF",
        "Milvus",
        "Milvus",
        "FAISS ou Milvus",
        "FAISS"
    ],
    "> 10M vecteurs": [
        "Milvus/Pinecone",
        "Milvus optimisé",
        "Milvus",
        "Milvus",
        "Milvus distributed",
        "Milvus",
        "Milvus (DevOps)"
    ]
})

print("\n" + "="*100)
print("MATRICE DE DÉCISION")
print("="*100)
print(decision_matrix.to_string(index=False))

## 9. Considérations Production

### 9.1 Persistance et Sauvegarde

In [ ]:
# Sauvegarder un index FAISS
import pickle

def save_faiss_index(index: FAISSIndex, filepath: str):
    """Sauvegarder l'index FAISS sur disque"""
    # Sauvegarder l'index FAISS
    faiss.write_index(index.index, f"{filepath}.index")
    
    # Sauvegarder les documents
    with open(f"{filepath}.docs.pkl", 'wb') as f:
        pickle.dump(index.documents, f)
    
    print(f"✅ Index sauvegardé: {filepath}")

def load_faiss_index(filepath: str, dimension: int, index_type: str) -> FAISSIndex:
    """Charger un index FAISS depuis le disque"""
    index_obj = FAISSIndex(dimension, index_type)
    
    # Charger l'index FAISS
    index_obj.index = faiss.read_index(f"{filepath}.index")
    
    # Charger les documents
    with open(f"{filepath}.docs.pkl", 'rb') as f:
        index_obj.documents = pickle.load(f)
    
    print(f"✅ Index chargé: {filepath}")
    return index_obj

# Exemple de sauvegarde
save_faiss_index(faiss_indexes['hnsw'], 'index_hnsw')

# Exemple de chargement
loaded_index = load_faiss_index('index_hnsw', embeddings.shape[1], 'hnsw')
print(f"Index chargé: {loaded_index.get_stats()}")

### 9.2 Monitoring et Métriques

In [ ]:
class VectorDBMonitor:
    """Classe pour monitorer les performances en production"""
    
    def __init__(self):
        self.queries = []
        self.latencies = []
        self.results_counts = []
    
    def log_query(self, query: str, latency: float, num_results: int):
        self.queries.append(query)
        self.latencies.append(latency)
        self.results_counts.append(num_results)
    
    def get_metrics(self) -> Dict:
        if not self.latencies:
            return {}
        
        return {
            "total_queries": len(self.queries),
            "avg_latency_ms": np.mean(self.latencies),
            "p50_latency_ms": np.percentile(self.latencies, 50),
            "p95_latency_ms": np.percentile(self.latencies, 95),
            "p99_latency_ms": np.percentile(self.latencies, 99),
            "max_latency_ms": np.max(self.latencies),
            "avg_results": np.mean(self.results_counts)
        }
    
    def print_report(self):
        metrics = self.get_metrics()
        print("\n" + "="*60)
        print("RAPPORT DE MONITORING")
        print("="*60)
        for key, value in metrics.items():
            print(f"{key}: {value:.2f}" if isinstance(value, float) else f"{key}: {value}")

# Exemple d'utilisation
monitor = VectorDBMonitor()

# Simuler des requêtes
for query in query_embeddings:
    distances, indices, latency = faiss_indexes['hnsw'].search(query.reshape(1, -1), k=10)
    monitor.log_query("test query", latency, len(indices[0]))

monitor.print_report()

## 10. Conclusion et Bonnes Pratiques

### ✅ Points clés à retenir

1. **FAISS vs Milvus**:
   - FAISS: Idéal pour prototypes et datasets < 10M vecteurs
   - Milvus: Nécessaire pour scale et persistence en production

2. **Trade-offs**:
   - Latence vs Recall: IVF/HNSW offrent 95-98% recall avec 10-100x speedup
   - Mémoire vs Performance: HNSW utilise plus de RAM mais est plus rapide

3. **Métriques importantes**:
   - Latence P95/P99 (pas seulement moyenne)
   - Recall@k pour mesurer la qualité
   - Throughput (requêtes/seconde)

4. **Production**:
   - Monitoring continu des performances
   - Sauvegarde régulière des index
   - Tests de charge avant déploiement
   - Considérer les mises à jour incrémentales

### 📚 Ressources

- [FAISS Wiki](https://github.com/facebookresearch/faiss/wiki)
- [Milvus Documentation](https://milvus.io/docs)
- [Vector Database Comparison](https://benchmark.vectorview.ai/)
- [HNSW Paper](https://arxiv.org/abs/1603.09320)

## 📝 Exercices

### Exercice 1: Optimiser les paramètres
1. Testez différentes valeurs de `nprobe` pour IVF (5, 10, 20, 50)
2. Mesurez l'impact sur latence et recall
3. Trouvez le meilleur compromis

### Exercice 2: Scalabilité
1. Créez un dataset de 100K vecteurs
2. Comparez les temps d'indexation
3. Mesurez la dégradation de performance

### Exercice 3: Production-ready API
1. Créez une API FastAPI avec un index FAISS
2. Ajoutez du monitoring des latences
3. Implémentez un cache pour requêtes fréquentes

### Exercice 4: Milvus deployment
1. Déployez Milvus avec Docker Compose
2. Indexez 50K vecteurs
3. Comparez avec FAISS en termes de latence et features